In [1]:
# 📦 Importa librerías
import pandas as pd
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import mlflow
import os

# 📁 Configura rutas
DATA_PATH = "/home/user/proyecto-nitro/notebooks/data/enhanced_predictions.csv"
REPORTS_PATH = "/home/user/proyecto-nitro/notebooks/reports/time_series_example.png"  # Ruta exacta especificada

# ✅ Carga el DataFrame
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print(f"📄 CSV cargado con {df.shape[0]} filas y {df.shape[1]} columnas.")
    print("\n🔍 Primeras filas:")
    print(df.head(3))
else:
    raise FileNotFoundError(f"⚠️ Archivo no encontrado: {DATA_PATH}")

# 🧾 Verifica columna objetivo
target_column = "final_prediction"  # Cambiado a columna existente en tus datos
if target_column not in df.columns:
    available_cols = [col for col in df.columns if col not in ['sensor_id', 'value_last']]
    raise ValueError(f"⚠️ Columna objetivo no encontrada. Columnas disponibles: {available_cols}")

# 🔍 Selecciona features numéricas (excluyendo metadatos)
exclude_cols = ['sensor_id', 'value_last', 'unsupervised_anomaly', 'final_prediction']
feature_columns = [col for col in df.select_dtypes(include=['number']).columns 
                  if col not in exclude_cols and not col.startswith('iso_')]

if not feature_columns:
    raise ValueError("⚠️ No se encontraron columnas numéricas para features")

X = df[feature_columns]
y = df[target_column]

print(f"\n🔧 Features seleccionadas ({len(feature_columns)}):")
print(feature_columns)

# 🧠 Entrena el modelo (con parámetros básicos)
prev_model = RandomForestClassifier(n_estimators=50, random_state=42, max_depth=5)
prev_model.fit(X, y)

print("\n✅ Modelo entrenado:")
print(f"- Accuracy promedio (train): {prev_model.score(X, y):.2f}")

# 🧠 SHAP Analysis (optimizado para rendimiento)
sample_size = min(100, X.shape[0])  # Limita a 100 muestras para eficiencia
X_sample = X.iloc[:sample_size]

explainer = shap.TreeExplainer(prev_model)
shap_values = explainer.shap_values(X_sample)

print(f"\n📊 Generando gráfico SHAP con {sample_size} muestras...")

# 🎨 Gráfico mejorado
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
plt.title("Importancia de Features (SHAP Values)", fontsize=14, pad=20)
plt.xlabel("Impacto promedio en la predicción", fontsize=12)

# 🖼️ Guardado en ruta específica
plt.savefig(REPORTS_PATH, dpi=300, bbox_inches='tight')
plt.close()

print(f"✅ Gráfico guardado en: {REPORTS_PATH}")

# 🔄 MLflow Tracking (opcional)
try:
    mlflow.start_run()
    
    # Log de parámetros
    mlflow.log_params({
        "model_type": "RandomForest",
        "features": str(feature_columns),
        "target": target_column
    })
    
    # Log de artefactos
    if os.path.exists(REPORTS_PATH):
        mlflow.log_artifact(REPORTS_PATH)
        print("📊 Artefacto loggeado en MLflow")
    else:
        print("⚠️ No se pudo encontrar el gráfico para MLflow")
        
    mlflow.end_run()
except Exception as e:
    print(f"⚠️ Error en MLflow: {str(e)}")

print("\n🎉 Proceso completado!")

📄 CSV cargado con 5 filas y 7 columnas.

🔍 Primeras filas:
  sensor_id  value_last  unsupervised_anomaly  model_probability  \
0  sensor_0       20.55                     0              0.435   
1  sensor_1       20.72                     0              0.435   
2  sensor_2       28.36                     0              0.435   

   combined_probability  final_prediction  predicted_label  
0                0.2175                 0                0  
1                0.2175                 0                0  
2                0.2175                 0                0  

🔧 Features seleccionadas (3):
['model_probability', 'combined_probability', 'predicted_label']

✅ Modelo entrenado:
- Accuracy promedio (train): 1.00

📊 Generando gráfico SHAP con 5 muestras...
✅ Gráfico guardado en: /home/user/proyecto-nitro/notebooks/reports/time_series_example.png
📊 Artefacto loggeado en MLflow

🎉 Proceso completado!


<Figure size 1200x600 with 0 Axes>